In [110]:
from urllib import request
import bs4
import os, shutil
import string

In [111]:
base_url = "https://en.wikipedia.org/wiki/Category:Machine_learning"

def download_page(url):
    page = request.urlopen(url)
    return page.read().decode('utf-8')

page = download_page(base_url)

In [112]:
def get_urls(html):
    subs_href = []
    pages_href = []

    soup = bs4.BeautifulSoup(html, "html.parser")
    
    #subs = soup.find(id="mw-subcategories")
    #if subs is not None:
    #    subs = subs.findAll("div", {"class":'mw-category-group'})
    #    for sub in subs:
    #        for a in sub.findAll('a'):
    #            subs_href.append(a.get('href'))
    subs = soup.find(id="mw-subcategories")
    if subs is not None:
        subs = subs.findAll("div", {"class":'CategoryTreeItem'})
        for sub in subs:
            for a in sub.findAll('a'):
                subs_href.append(a.get('href'))

    pages = soup.find(id="mw-pages")
    if pages is not None:
        pages = pages.findAll("div", {"class":'mw-category-group'})
        for page in pages:
            for a in page.findAll('a'):
                pages_href.append(a.get('href'))
            
    return dict(pages=pages_href, subcategories=subs_href)

In [120]:
wiki_url = "https://en.wikipedia.org"

def crawl(url):
    pages = []
    crawled_subs = []
    
    uncrawled_subs = []
    depths = []
    
    max_depth = 3
    
    page = download_page(url)
    print("Crawling url: %s", url)
    url_list = get_urls(page)
    uncrawled_subs += url_list['subcategories']
    depths += [1]* len(uncrawled_subs)
    pages += url_list['pages']
    
    while uncrawled_subs != []:
        print("Pending crawl:", len(uncrawled_subs))
        u = uncrawled_subs.pop(0)
        depth = depths.pop(0)
        if u not in crawled_subs:
            print("Crawling url: %s" % wiki_url+u)
            p = download_page(wiki_url+u)
            u_list = get_urls(p)
            print("Found subcats:", u_list['subcategories'])
            crawled_subs.append(u)
            if depth + 1 <= max_depth:
                print("Appending subcats:", u_list['subcategories'], "with depth", depth+1)
                uncrawled_subs += u_list['subcategories']
                depths+=[depth+1]*len(u_list['subcategories'])
            pages += u_list['pages']
           
    return {'pages':pages, 'subcategories':crawled_subs}

wiki_urls = crawl(base_url)

Crawling url: %s https://en.wikipedia.org/wiki/Category:Machine_learning
Pending crawl: 27
Crawling url: https://en.wikipedia.org/wiki/Category:Applied_machine_learning
Found subcats: []
Appending subcats: [] with depth 2
Pending crawl: 26
Crawling url: https://en.wikipedia.org/wiki/Category:Artificial_neural_networks
Found subcats: ['/wiki/Category:Deep_learning', '/wiki/Category:Neural_network_software']
Appending subcats: ['/wiki/Category:Deep_learning', '/wiki/Category:Neural_network_software'] with depth 2
Pending crawl: 27
Crawling url: https://en.wikipedia.org/wiki/Category:Bayesian_networks
Found subcats: []
Appending subcats: [] with depth 2
Pending crawl: 26
Crawling url: https://en.wikipedia.org/wiki/Category:Classification_algorithms
Found subcats: ['/wiki/Category:Artificial_neural_networks', '/wiki/Category:Decision_trees', '/wiki/Category:Ensemble_learning']
Appending subcats: ['/wiki/Category:Artificial_neural_networks', '/wiki/Category:Decision_trees', '/wiki/Category:

In [121]:
with open("wp_ml_urls.txt", "a") as url_file:
    for url in wiki_urls['pages']:
        url_file.writelines(url + "\n")

In [115]:
#Create a folder to store downloaded html files
if os.path.lexists("raw_text"):
    shutil.rmtree("raw_text")
os.mkdir(os.getcwd() + "/" + "raw_text")

In [119]:
base="https://en.wikipedia.org"
def edit_name(s):
    return ''.join([letter for letter in s if letter.isalnum()])
     
def download_article(url):
    page=download_page(url)
    soup=bs4.BeautifulSoup(page, "html.parser")
    soup=soup.find(id="mw-content-text")
    p_list=list(soup.findAll('p'))
    article=' '.join(map(str, p_list))
    soup = bs4.BeautifulSoup(article, "html.parser")
    article=soup.get_text()
    return article
    
inum = 0
fnum = len(urls)
with open("wp_ml_urls.txt","r") as urls:
    for line in urls:
        print(line)
        article=download_article(base+line)
        with open("raw_text/"+edit_name(line[6:]),"w") as text:
            text.write(article)
        inum+=1
        print("Processing", line, str(round((inum/fnum)*100, 2))+"%")

/wiki/Machine_learning

/wiki/Accuracy_paradox

/wiki/Action_model_learning

/wiki/Active_learning_(machine_learning)

/wiki/Adversarial_machine_learning

/wiki/AIXI

/wiki/Algorithmic_inference

/wiki/AlphaGo

/wiki/Apprenticeship_learning

/wiki/Bag-of-words_model

/wiki/Ball_tree

/wiki/Base_rate

/wiki/Bayesian_interpretation_of_kernel_regularization

/wiki/Bayesian_optimization

/wiki/Bayesian_structural_time_series

/wiki/Bias%E2%80%93variance_tradeoff

/wiki/Binary_classification

/wiki/Bing_Predicts

/wiki/Bongard_problem

/wiki/Bradley%E2%80%93Terry_model

/wiki/Catastrophic_interference

/wiki/Category_utility

/wiki/CBCL_(MIT)

/wiki/CIML_community_portal

/wiki/Cleverbot

/wiki/Cognitive_robotics

/wiki/Committee_machine

/wiki/Draft:Complexit%C3%A9_de_Rademacher

/wiki/Computational_learning_theory

/wiki/Concept_drift

/wiki/Concept_learning

/wiki/Conditional_random_field

/wiki/Confusion_matrix

/wiki/Constrained_conditional_model

/wiki/Coupled_pattern_learner

/wiki/C

#### NLTK

In [122]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [138]:
#Create a folder to store downloaded html files
if os.path.lexists("processed_text"):
    shutil.rmtree("processed_text")
os.mkdir(os.getcwd() + "/" + "processed_text")

wnl = WordNetLemmatizer()

fnum = len(os.listdir("raw_text"))
inum = 0

for fn in list(os.listdir("raw_text")):
    with open("raw_text/"+fn,"r") as file:
        
        inputs=file.read()
        input_tokenized=nltk.wordpunct_tokenize(inputs)
        input_tokenized = [el.lower() for el in input_tokenized if len(el) > 3]
        input_tokenized = [el for el in input_tokenized if el not in string.punctuation]
        stop=stopwords.words("english")
        input_tokenized = [el for el in input_tokenized if el not in stop]
        input_tokenized = [t[0] for t in nltk.pos_tag(input_tokenized) if t[1] not in ['IN', 'DT']]
        
        # Remove unwanted words
        ignore_words = ["displaystyle"]
        input_tokenized = [el for el in input_tokenized if el not in ignore_words]
        
        input_tokenized = [wnl.lemmatize(t) for t in input_tokenized]
        
        with open("processed_text/"+fn, "w") as out:
            out.write(' '.join(input_tokenized))
        
        inum += 1
        print("Processing", fn, str(round((inum/fnum)*100, 2))+"%")

Processing 12BCEB5approximatenearestneighborsearch 0.1%
Processing AAAIConferenceonArtificialIntelligence 0.2%
Processing AbsorbingMarkovchain 0.3%
Processing Accuracyparadox 0.4%
Processing Actionmodellearning 0.5%
Processing Activationfunction 0.6%
Processing Activelearningmachinelearning 0.7%
Processing Activityrecognition 0.81%
Processing AdaBoost 0.91%
Processing ADALINE 1.01%
Processing Adaptiveneurofuzzyinferencesystem 1.11%
Processing Adaptiveresonancetheory 1.21%
Processing Adaptivesampling 1.31%
Processing Additivesmoothing 1.41%
Processing Adjustedmutualinformation 1.51%
Processing Adversarialmachinelearning 1.61%
Processing Affinitypropagation 1.71%
Processing AI50 1.81%
Processing AIXI 1.91%
Processing AlbertoBroggi 2.01%
Processing AlchemyAPI 2.11%
Processing Algorithmiccomposition 2.22%
Processing Algorithmicinference 2.32%
Processing Algorithmiclearningtheory 2.42%
Processing AlmeidaE28093Pinedarecurrentbackpropagation 2.52%
Processing ALOPEX 2.62%
Processing AlphaGo 2.

In [139]:
# Running LDA
import gensim

In [140]:
wikis = []

for fn in list(os.listdir("processed_text")):
    with open("processed_text/"+fn,"r") as file:
        contents = file.read()
        wikis.append(contents.split(' '))

corp_dict = gensim.corpora.Dictionary(wikis)
corp_dict.save("wiki-dict")

In [141]:
wiki_bow = [corp_dict.doc2bow(e) for e in wikis]

In [145]:
gensim.corpora.MmCorpus.serialize("wiki-dict", wiki_bow)

In [153]:
lda=gensim.models.LdaModel(wiki_bow, num_topics=5)

In [158]:
def get_topic_terms_words(i):
    terms = lda.get_topic_terms(i)
    return [(corp_dict.get(t[0]), t[1]) for t in terms]

get_topic_terms_words(3)

[('algorithm', 0.0075653550315541093),
 ('function', 0.0074454148900689517),
 ('model', 0.0071753226225463786),
 ('data', 0.0063401841430831855),
 ('used', 0.0056487611634106431),
 ('variable', 0.0054726913815499687),
 ('value', 0.0041517402889358112),
 ('network', 0.003979730631122628),
 ('example', 0.0039151654961155401),
 ('problem', 0.0038308731614008192)]